In [0]:
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.layers import Dropout
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping
import pandas as pd
import time
from sklearn import preprocessing
from sklearn.model_selection import KFold
import os
import tensorflow as tf
from keras import regularizers

tf.compat.v1.disable_eager_execution()
from tensorflow.compat.v1 import ConfigProto, Session
from tensorflow.compat.v1 import InteractiveSession

drive_path = "Stead_Classification/"
outPath = drive_path +"FCN/Normalizers_Sample"

train_data = np.load(drive_path +'train-BH-7500*2.npy')
test_data = np.load(drive_path +'test-BH-374*2.npy')

# Shuffle data
np.random.shuffle(train_data)
np.random.shuffle(test_data)

x_data = train_data[:,1:]
y_data = train_data[:,0]
x_test = test_data[:,1:]
y_test = test_data[:,0]

# Sample Standardization
x_data = [(x-np.mean(x))/np.std(x) for x in x_data]
x_test = [(x-np.mean(x))/np.std(x) for x in x_test]

# Reshape data
x_data = np.array(x_data)
x_test = np.array(x_test)
x_data = x_data.reshape(x_data.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

nb_epochs = 100
kf_val = 5
batch_list = [1,2,4,8,16,32,64,128,256,512]

for batch_size in batch_list:
    test = []
    train = []
    val = []
    print("###################### Batch = " +str(batch_size) +" ######################")

    ################ Cross-Val #####################
    kf = KFold(n_splits=kf_val, shuffle=False)
    itr = 0
    for train_idx, val_idx in kf.split(x_data):
        start_time = time.time()
        f = open(outPath +"/stat.txt", "a+")
        itr += 1
        print("********* Iteration " +str(itr) +" *********")

        x_train, y_train = x_data[train_idx], y_data[train_idx]
        x_val, y_val = x_data[val_idx], y_data[val_idx]
        nb_classes = len(np.unique(y_val))
        Y_train = np_utils.to_categorical(y_train, nb_classes)
        Y_val = np_utils.to_categorical(y_val, nb_classes)
        Y_test = np_utils.to_categorical(y_test, nb_classes)
        
        x = keras.layers.Input(x_train.shape[1:])
        conv1 = keras.layers.Conv1D(64, 5, padding='same', kernel_initializer=keras.initializers.glorot_uniform())(x)
        conv1 = keras.layers.normalization.BatchNormalization()(conv1)
        conv1 = keras.layers.Activation('relu')(conv1)

        conv2 = keras.layers.Conv1D(128, 5, padding='same', kernel_initializer=keras.initializers.glorot_uniform())(conv1)
        conv2 = keras.layers.normalization.BatchNormalization()(conv2)
        conv2 = keras.layers.Activation('relu')(conv2)

        conv3 = keras.layers.Conv1D(128, 3, padding='same', kernel_initializer=keras.initializers.glorot_uniform())(conv2)
        conv3 = keras.layers.normalization.BatchNormalization()(conv3)
        conv3 = keras.layers.Activation('relu')(conv3)

        conv4 = keras.layers.Conv1D(64, 3, padding='same', kernel_initializer=keras.initializers.glorot_uniform())(conv3)
        conv4 = keras.layers.normalization.BatchNormalization()(conv4)
        conv4 = keras.layers.Activation('relu')(conv4)

        full = keras.layers.pooling.GlobalAveragePooling1D()(conv4)
        out = keras.layers.Dense(nb_classes, activation='softmax', activity_regularizer=regularizers.l2(1e-5))(full)
        model = Model(inputs=x, outputs=out)

        optimizer = keras.optimizers.Adam()
        model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        path_checkpoint = outPath +'/checkpoint-batch=' +str(batch_size) +'-' +str(itr) +'.keras'
        callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                        monitor='val_loss',
                                        verbose=0,
                                        save_weights_only=True,
                                        save_best_only=True)
        callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,min_lr=0,patience=10,verbose=0)
        callback_tensorboard = TensorBoard(log_dir= outPath +'/log-batch=' +str(batch_size) +'-' +str(itr),
                                histogram_freq=0,
                                write_graph=False,
                                write_images=False)
        callback_early_stopping = EarlyStopping(monitor='val_loss', patience=30, verbose=1)

        hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
                verbose=1, validation_data=(x_val, Y_val),
                callbacks=[callback_checkpoint, callback_early_stopping, callback_reduce_lr, callback_tensorboard])
        log = pd.DataFrame(hist.history)

    ################### TEST DATA
        tr = 0
        fl = 0
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        try:
            model.load_weights(path_checkpoint)
        except Exception as error:
            print("Error trying to load checkpoint.")
            print(error)

        y_predicted = model.predict(x_test)

        for i in range(len(Y_test)):
            if  np.argmax(Y_test[i]) == np.argmax(y_predicted[i]):
                tr += 1
                if np.argmax(Y_test[i]) == 0:
                    tn += 1
                elif np.argmax(Y_test[i]) == 1:
                    tp += 1

            else:
                fl += 1
                if np.argmax(Y_test[i]) == 0:
                    fn += 1
                elif np.argmax(Y_test[i]) == 1:
                    fp += 1
        test.append(float(tr)/(tr+fl))
        np.save(outPath +'/loss-'+str(batch_size)+'-'+str(itr), hist.history['loss'])
        np.save(outPath +'/val-loss-'+str(batch_size)+'-'+str(itr), hist.history['val_loss'])
        np.save(outPath +'/acc-'+str(batch_size)+'-'+str(itr), hist.history['accuracy'])
        np.save(outPath +'/val-acc-'+str(batch_size)+'-'+str(itr), hist.history['val_accuracy'])
        plt.plot(hist.history['loss'], label="Train")
        plt.plot(hist.history['val_loss'], label="Validation")
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend()
        plt.savefig(outPath +'/Loss-batch='+str(batch_size)+'-'+str(itr) +'.png', dpi=600)
        plt.close()
        plt.plot(hist.history['accuracy'], label="Train")
        plt.plot(hist.history['val_accuracy'], label="Validation")
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend()
        plt.savefig(outPath +'/Accuracy-batch='+str(batch_size)+'-'+str(itr) +'.png', dpi=600)
        plt.close()
        
        train.append(log.loc[log['loss'].idxmin]['accuracy'])
        val.append(log.loc[log['loss'].idxmin]['val_accuracy'])
        f.write("train: " +str(log.loc[log['loss'].idxmin]['accuracy']) +"\tval: " +str(log.loc[log['loss'].idxmin]['val_accuracy']) +"\ttest: " +str(float(tr)/(tr+fl)) +"\n")
        f.write("tp: " +str(tp) +"\tfp: " +str(fp) +"\ttn: " +str(tn) +"\tfn: " +str(fn) +"\n")
        f.write("--- " +str(time.time() - start_time) +" seconds ---\n")
        print("\n--- " +str(time.time() - start_time) +" seconds ---\n")
        f.close()
        
    f = open(outPath +"/stat.txt", "a+")
    f.write("FOR BATCH = " +str(batch_size) +" (train/val/test) ==> " +str(np.asarray(train).mean()) +"/" +str(np.asarray(val).mean()) +"/" +str(np.asarray(test).mean()) +"\n")
    f.write("std train = " +str(np.std(np.asarray(train))) +"\tstd val = " +str(np.std(np.asarray(val))) +"\tstd test = " +str(np.std(np.asarray(test))) +"\n")
    f.close()
    plt.plot(train, label="train")
    plt.plot(val, label="val")
    plt.plot(test, label="test")
    plt.legend()
    # plt.show()
    plt.title("batch=" +str(batch_size))
    plt.savefig(outPath +"/batch=" +str(batch_size) +'.png', dpi=600)
    plt.close()
